In [1]:
# Data handling imports

import yfinance as yf
import pandas as pd
import numpy as np
# import datetime as dt
from datetime import date
# import datetime

In [2]:
# Webscraping imports

import json
import re

import requests 
from bs4 import BeautifulSoup 

In [3]:
tix = pd.read_table('./DJIA_comps.txt', header=None).to_numpy()

In [4]:
tix = np.concatenate(tix, axis=0)

In [5]:
# # Attributes

static_attributes = ['symbol','exchange', 'sector', 'industry', 'type' ]
daily_attributes = ['date','trending', 'trendingScore', 'sentimentChange', 'volumeChange', 'price', 'change',
                    'percent', 'lastUpdated', 'extendedHoursChange', 'open', 'outcome', 'last', 'extendedHoursPrice',
                    'easternDateTime', 'previousCloseDate', 'previousClose', 'dateTime', 'volume', 'lastSize', 'low',
                    'percentChange', 'high', 'extendedHoursPercentChange']#, 'extendedHoursDateTime']



# # Initializing pandas dataframe for static data

# attr_df = pd.DataFrame(columns=static_attributes, index=tix)

# # Initializing pandas dataframes for daily data

# tix_df_list = []

# for tick in tix:
#     globals()['{}_df'.format(tick)] = pd.DataFrame(columns=daily_attributes, index=np.arange(100))
#     tix_df_list.append(globals()['{}_df'.format(tick)])
    
# tix_df = [MMM_df, AXP_df, AAPL_df, BA_df, CAT_df, CVX_df, CSCO_df, KO_df, DIS_df, DOW_df, XOM_df, 
#       GS_df, HD_df, IBM_df, INTC_df, JNJ_df, JPM_df, MCD_df, MRK_df, MSFT_df, NKE_df, PFE_df, PG_df,
#       TRV_df, RTX_df, UNH_df, VZ_df, V_df, WMT_df, WBA_df]

In [6]:
# Load previous data on current day at 8:00pm or later

import pickle

with open('tix_df.pkl', 'rb') as f:
    tix_df = pickle.load(f)

In [22]:
# for i in range(len(tix_df)):
#     tix_df[i].drop(tix_df[i].index[6], inplace=True)
#     tix_df[i].reset_index(drop=True, inplace=True)


In [7]:
with open('incrementer.pkl', 'rb') as g:
    daily_row = pickle.load(g)
    
daily_row += 1

In [8]:
daily_row

6

In [80]:
for j in range(len(tix)):
#for tick in tix:
    symbol = tix[j]
    url = "https://stocktwits.com/symbol/{}".format(symbol)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html5lib')
    
    script = soup.find_all("script")
    pattern = re.compile('window.INITIAL_STATE = {.*}')
    
    for i in script:
        strObj = i.text
        match = pattern.search(strObj)
        if match:
            start_str = strObj.find('window.INITIAL_STATE')
            front_str = strObj[start_str:] # front is proper
            #end_str = front_str.find(';') # find end
            # Updated end of string identification on 6/8/2020
            final_str = '"}}}'
            end_str = front_str.find(final_str)
            good_str = front_str[:end_str+4]
            jsonString = good_str.split("window.INITIAL_STATE = ")[1][:].encode('utf8').decode('unicode_escape')
            jsonData = json.loads(jsonString, strict=False)

        
        # LOOP THROUGH STATIC AND DAILY ATTRIBUTES TO BUILD OUT PANDAS DATAFRAME USING FORMAT BELOW
    
    #print(jsonData)
    
#     for static in static_attributes:
#         attr_df.loc["{}".format(symbol), static] = jsonData['stocks']['inventory'] \
#                                                             ['{}'.format(symbol)][static]

    # Change this everyday, 0 = 5/31/2020
#     daily_row = 1
    # could load this into a pickle

#     print(daily_row)
    
#     for df in tix_df_list:
#         #print(daily_row)
    z = 1
    for daily in daily_attributes:
        if daily == "date":
            tix_df[j].iloc[daily_row, 0] = date.today()
        else:
            tix_df[j].iloc[daily_row, z] = jsonData['stocks']['inventory'] \
                                                        ['{}'.format(symbol)][daily]
            z += 1



In [88]:
tix_df[-1]

,date,trending,trendingScore,sentimentChange,volumeChange,price,change,percent,lastUpdated,extendedHoursChange,...,previousCloseDate,previousClose,dateTime,volume,lastSize,low,percentChange,high,extendedHoursPercentChange,extendedHoursDateTime
0,2020-05-31,False,-1.82496,-3.69,-2.17,43.03,1.2,2.864,"05/29/2020, 07:59:24 PM EDT",0.09,...,2020-05-28,41.83,2020-05-29 23:59:24,8858013,2694889,41.23,2.864,42.97,0.21,2020-05-29 23:59:24
1,2020-06-01,False,-0.27981,0.31,10.11,42.91,-0.03,-0.063,"06/01/2020, 07:40:07 PM EDT",-0.39,...,2020-05-29,42.94,2020-06-01 23:40:07,5772089,474574,42.6,-0.063,43.48,-0.901,2020-06-01 23:40:07
2,2020-06-02,False,2.02319,-3.76,-13.27,43.67,0.37,0.853,"06/02/2020, 07:46:41 PM EDT",0.12,...,2020-06-01,43.3,2020-06-02 23:46:41,4284171,659369,43.13,0.853,43.87,0.276,2020-06-02 23:46:41
3,2020-06-03,False,3.6671,-11.31,-8.24,43.68,0.13,0.299,"06/03/2020, 07:59:58 PM EDT",0.07,...,2020-06-02,43.55,2020-06-03 23:59:58,4681745,527441,43.45,0.299,44.41,0.161,2020-06-03 23:59:58
4,2020-06-04,False,0.533871,-10.26,8.64,44.17,0.56,1.285,"06/04/2020, 07:55:17 PM EDT",-0.03,...,2020-06-03,43.61,2020-06-04 23:55:17,4949552,437762,42.77,1.285,44.365,-0.068,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
import pickle

with open('./tix_df.pkl', 'wb') as f:
    pickle.dump(tix_df, f)

In [90]:
# with open('./attr_df.pkl', 'wb') as g:
#     pickle.dump(attr_df, g)

In [91]:
with open('./incrementer.pkl', 'wb') as g:
    pickle.dump(daily_row, g)

In [136]:
# GOOD BASE

# symbol = "AXP"

# url = "https://stocktwits.com/symbol/{}".format(symbol)

# r = requests.get(url) 
  
# soup = BeautifulSoup(r.content, 'html5lib') 
# # print(soup.prettify())

In [137]:
# # GOOD BASE

# script = soup.find_all("script")
# pattern = re.compile('window.INITIAL_STATE = {.*}')#'#JSON.parse(.*);')

# for i in script:
#     strObj = i.text
#     match = pattern.search(strObj)
#     #print(match)
#     if match:
#         start_str = strObj.find('window.INITIAL_STATE')
#         front_str = strObj[134:] # front is proper
#         end_str = front_str.find(';') # find end
#         good_str = front_str[:end_str]
#         jsonString = good_str.split("window.INITIAL_STATE = ")[1][:].encode('utf8').decode('unicode_escape')
        
#         #print(good_str.split()[1:-1].encode('utf8'))
#         #jsonString = good_str.encode('utf8').decode('unicode_escape')
# #         print(jsonString)
# #         print(jsonString[4040])
#         jsonData = json.loads(jsonString, strict=False)
#         print(jsonData)

{'ads': {'awaitingRefresh': False, 'channel': '', 'initialized': False, 'queue': [], 'pathname': ''}, 'analytics': {'Context': '', 'pathname': '', 'Platform': '', 'user': {}}, 'announcements': {'latest': {}, 'dismissed': []}, 'advancedSearch': {'countsFetchingState': 'initial', 'fetchingState': 'initial', 'messages': [], 'symbolCounts': [], 'total': 0}, 'content': {}, 'currentUser': {'directMessages': {'isFetching': False, 'cursor': {}, 'items': {}, 'unread': 0}, 'followSuggestions': {'fetchError': False, 'isFetching': False, 'userIds': [], 'users': {}}, 'info': {'isAuthenticated': False}, 'notifications': {'cursor': {}, 'isFetching': False, 'items': {}, 'itemIds': [], 'listType': 'NotificationList', 'unread': 0}, 'rooms': {'allRooms': {}}, 'roomsNotifications': {'allUnreadCount': 0, 'notifications': {}}}, 'form': {}, 'messages': {'pathname': None}, 'messagePrices': {}, 'modal': {'chart': None, 'conversation': {'cursor': {}, 'messages': [], 'parent': {}}, 'direct': {'cursor': {}, 'conv

In [140]:
# jsonData['stocks']['inventory']['AXP']

{'fundamentals': {},
 'sentimentData': {'data': [], 'isFetching': True},
 'volumeData': {'data': [], 'isFetching': True},
 'symbol': 'AXP',
 'id': 4423,
 'title': 'American Express Company',
 'aliases': [],
 'isFollowing': False,
 'watchlistCount': 12408,
 'exchange': 'NYSE',
 'sector': 'Financial',
 'industry': 'Credit Services',
 'logoUrl': 'http://logos.xignite.com/NYSE/00003630.gif',
 'trending': False,
 'trendingScore': -3.21555,
 'sentimentChange': 0,
 'volumeChange': -1.34,
 'price': 96,
 'change': -2.06,
 'percent': -2.0709999999999997,
 'lastUpdated': '05/29/2020, 07:59:59 PM EDT',
 'extendedHoursChange': 0.93,
 'extendedHoursType': 'PostMarket',
 'open': 95.5,
 'outcome': 'Success',
 'last': 96,
 'extendedHoursPrice': 96,
 'easternDateTime': '05/29/2020, 04:00:00 PM EDT',
 'previousCloseDate': '2020-05-28',
 'previousClose': 98.06,
 'dateTime': '2020-05-29 23:59:59',
 'volume': 7711888,
 'lastSize': '1123602',
 'low': 94.66,
 'percentChange': -2.0709999999999997,
 'high': 97.